In [ ]:
# import os
# from copy import deepcopy
# import json

# import requests
# import numpy as np

# # Azure Services
# from azure.ai.vision.imageanalysis import ImageAnalysisClient
# from azure.ai.vision.imageanalysis.models import VisualFeatures
# from azure.ai.formrecognizer import DocumentAnalysisClient
# from azure.core.credentials import AzureKeyCredential

# # Visualization
# from PIL import Image
# import matplotlib.pyplot as plt

# from pprint import pprint

# Setup

In [ ]:
# plt.rcParams['figure.figsize'] = (2,2)

In [ ]:
# TEMPLATE = {
#     'status': None,
#     'img_location': None,
#     "applied_fuctions": [],
#     'n_info': 0,
#     'vehicle_type': {
#         'label': None,
#         'probability': None
#     },
#     'photo_type': {
#         'label': None,
#         'probability': None
#     },
#     'vin': {
#         'text': None,
#         'probability': None
#     },
#     'plate': {
#         'text': None,
#         'probability': None
#     }
# }

In [ ]:
# VEHICLE_TYPES = ['Automóvil', 'Camión', 'Motocicleta']
# PHOTO_TYPES = [
#     'FRONTAL R2'  , 'FRONTAL R4'  ,
#     'TRASERA R2'  , 'TRASERA R4'  ,
#     'DERECHA R2'  , 'DERECHA R4'  ,
#     'IZQUIERDA R2', 'IZQUIERDA R4',
#     'INTERIOR R2' , 'INTERIOR R4' ,
#     'VIN',
#     'DOCUMENTO'
# ]
# # camara_angles = [
# #     'FRONTAL',
# #     'TRASERA',
# #     'DERECHA',
# #     'IZQUIERDA',
# #     'INTERIOR',
# # ]


## Azure Services

### Custom Vision

In [ ]:
# HEADERS_CUSTOM_VISION = {
#     'Content-Type': 'application/octet-stream',
#     'Prediction-Key': '5ceef9f635804c05ae36fdca6dd99eaf'
# }

# # Classification -------------------------------------------------------
# PROJECT_NAME = 'Iteration3%20%7C%20F2M2'
# PROJECT_ID = '077a7a83-5a6f-4a47-a946-374814e786f4'
# ENDPOINT_CLASSIFICATION = (
#     f'https://cstvtclasificadorimagentesdeveastus2-prediction.'
#     f'cognitiveservices.azure.com/customvision/v3.0/Prediction/'
#     f'{PROJECT_ID}/classify/iterations/{PROJECT_NAME}/image'
# )
# THRESHOLD_CLASSIFICATION = 0.7
# # ----------------------------------------------------------------------


# # Plate Recognition ----------------------------------------------------
# PROJECT_NAME = 'Iteration2%20%7C%20F2M1'
# PROJECT_ID = 'ca4fe2cf-23ff-44ab-8604-9a3b092caecf'
# ENDPOINT_PLATE_DETECTION = (
#     f'https://cstvtclasificadorimagentesdeveastus2-prediction.'
#     f'cognitiveservices.azure.com/customvision/v3.0/Prediction/'
#     f'{PROJECT_ID}/detect/iterations/{PROJECT_NAME}/image'
# )
# THRESHOLD_PLATE_DETECTION = 0.94
# # ----------------------------------------------------------------------

### Computer Vision

In [ ]:
# # Plate Number Extraction ----------------------------------------------
# ENDPOINT_PLATE_EXTRACTION = (
#     'https://cv-clasificadorimagenes-dev-eastus2-01.'
#     'cognitiveservices.azure.com/'
# )
# KEY_PLATE_EXTRACTION = 'a25a975c79e1418cbde0a23b4021b74e'
# # ----------------------------------------------------------------------

### Document Intelligence

In [ ]:
# ENDPOINT_DOCUMENT_INTELLIGENCE = 'https://eastus2.api.cognitive.microsoft.com/'
# KEY_DOCUMENT_INTELLIGENCE = '1bcc168accb740f09a59b7313db475a4'
# MODEL_ID = 'Iteracion2-neural'

# THRESHOLD_DOC_EXTRACTION = 0.5

# Functions

## Classification

In [ ]:
# def classify_image(
#     results: dict,
#     img_is_url: bool = False
#     ) -> dict:
#     """
#     Classifies an image using the Azure AI Custom Vision API and updates the 
#     results dictionary with the most probable predictions.

#     This function takes an image location, which can be either a local file 
#     path or a URL, and sends it to the Azure AI Custom Vision API for 
#     classification. The API returns multiple predictions, each associated 
#     with a probability score. The predictions are filtered based on a 
#     probability threshold (`THRESHOLD_CLASSIFICATION`) and sorted to ensure 
#     that the most probable predictions are included in the results. The 
#     function updates the results dictionary with the classified labels and 
#     their corresponding probabilities.

#     Parameters:
#     -----------
#     img_location : str
#         The file path or URL of the image to be classified.
#     results : dict
#         A dictionary template to store the classification results, by default
#         template.
#     img_is_url : bool, optional
#         Indicates whether `img_location` is a URL or a local file path, by 
#         default False.

#     Returns:
#     --------
#     dict
#         The updated results dictionary containing the classification labels 
#         and probabilities.
    
#     """
#     results['applied_fuctions'].append('classify_image')
#     img_location = results['img_location']

#     # Call Azure AI Custom vision API (multi-label) --------------------
#     if img_is_url:
#         pass

#     else:
#         with open(img_location, 'rb') as img_binary:
#             response = requests.request( 
#                 method="POST", 
#                 url=ENDPOINT_CLASSIFICATION,
#                 headers=HEADERS_CUSTOM_VISION, 
#                 data=img_binary
#             )
#     # ------------------------------------------------------------------

#     # The predictions are ordered from lowest to highest probability to 
#     # guarantee that the most probable values will be present in 
#     # 'results', regardless of the number of labels predicted. 
#     predictions = response.json()['predictions'][::-1]

#     for prediction in predictions:  
            
#         if prediction['probability'] >= THRESHOLD_CLASSIFICATION:

#             if prediction['tagName'] in VEHICLE_TYPES:
#                 results['vehicle_type']['label']       = prediction['tagName']
#                 results['vehicle_type']['probability'] = prediction['probability']

#             else:
#                 results['photo_type']['label']         = prediction['tagName'].split()[0]
#                 results['photo_type']['probability']   = prediction['probability']

#             results['n_info'] += 1

#     return results        


# # results1 = classify_image(IMG_PATH)
# # pprint(results1)

## Plate Detection

In [ ]:
# def crop_image(
#     img_location,
#     image,
#     bounding_box
#     ):

#     # Extract region defined by the bounding box
#     left   = bounding_box['left']   * image.width
#     top    = bounding_box['top']    * image.height
#     width  = bounding_box['width']  * image.width
#     height = bounding_box['height'] * image.height

#     # Crop the image
#     cropped_image = image.crop((
#         left, 
#         top, 
#         left + width, 
#         top + height
#     ))

#     # Plot the cropped image
#     plt.imshow(cropped_image)
#     plt.axis('off')
#     plt.savefig(
#         f'work-zone/Resultados/{os.path.basename(img_location)}', 
#         bbox_inches='tight', 
#         pad_inches = 0
#     )
#     plt.show()

#     return cropped_image



In [ ]:
# def detect_plate(
#     results: dict,
#     img_is_url: bool = False  
#     ) -> dict:
#     """
#     Detects and extracts license plates from an image using the Azure AI Custom 
#     Vision API for object detection.
#     """
#     img_location = results['img_location']
#     results['applied_fuctions'].append('detect_plate')

#     # Call Azure AI Custom vision API (object-detection) ---------------
#     if img_is_url:
#         pass
#     else:
#         with open(img_location, 'rb') as img_binary:
#             original_image = Image.open(img_location)
#             response = requests.request(
#                 method="POST", 
#                 url=ENDPOINT_PLATE_DETECTION, 
#                 headers=HEADERS_CUSTOM_VISION, 
#                 data=img_binary
#             )
#     # ------------------------------------------------------------------

#     predictions = response.json()['predictions']

#     if predictions and (predictions[0]['probability'] 
#                         >= THRESHOLD_PLATE_DETECTION
#                        ):
#         bounding_box = predictions[0]['boundingBox']
#         cropped_image = crop_image(
#             img_location, 
#             original_image, 
#             bounding_box
#         )

#     else:  # NO PLATE DETECTED
#         results['status'] = 'REVIEW'
#         cropped_image = None

#     return results, cropped_image 
        

# # results2, cropped_image = detect_plate(IMG_PATH, results=results1)
# # results2

## Plate Number Extraction

In [ ]:
# # Create an Image Analysis client
# client = ImageAnalysisClient(
#     endpoint=ENDPOINT_PLATE_EXTRACTION,
#     credential=AzureKeyCredential(KEY_PLATE_EXTRACTION)
# )

# # Load image to analyze into a 'bytes' object
# with open(f'work-zone/Resultados/{os.path.basename(IMG_PATH)}', "rb") as f:
#     image_data = f.read()

# # Extract text (OCR) from an image stream. This will be a synchronously (blocking) call.
# result = client.analyze(
#     image_data=image_data,
#     visual_features=[VisualFeatures.READ]
# )

# result_lines = result.read['blocks'][0]['lines']
# pprint(result_lines)


## VIN Extraction

In [ ]:
# def extract_text(
#     content_type: str, 
#     results: dict, 
#     img_is_url: bool
#     ):
#     """
#     docstring
#     """
#     img_location = results['img_location']
#     results['applied_fuctions'].append('extract_text')

#     # Call Azure AI Vision API (OCR) -----------------------------------
#     client = ImageAnalysisClient(
#         endpoint=ENDPOINT_PLATE_EXTRACTION,
#         credential=AzureKeyCredential(KEY_PLATE_EXTRACTION)
#     )

#     if img_is_url:
#         pass
#     else:
#         with open(img_location, "rb") as f:
#             image_data = f.read()

#         # Extract text (OCR) from an image stream. This will be a synchronously (blocking) call.
#         response = client.analyze(
#             image_data=image_data,
#             visual_features=[VisualFeatures.READ]
#         )
#     # ------------------------------------------------------------------
    
#     lines = response.read['blocks'][0]['lines']

#     results[content_type]['text'] = str(lines)

#     return results



In [ ]:
# # Create an Image Analysis client
# client = ImageAnalysisClient(
#     endpoint=ENDPOINT_PLATE_EXTRACTION,
#     credential=AzureKeyCredential(KEY_PLATE_EXTRACTION)
# )

# with open(IMG_PATH, "rb") as f:
#     image_data = f.read()

# # Extract text (OCR) from an image stream. This will be a synchronously (blocking) call.
# result = client.analyze(
#     image_data=image_data,
#     visual_features=[VisualFeatures.READ]
# )
# result_lines = result.read['blocks'][0]['lines']
# pprint(result_lines)

## Document Intelligence

In [ ]:
# def extract_from_doc(
#         results,
#         img_is_url
#         ):
#     """
#     Remember to remove the key from your code when you're done, and 
#     never post it publicly. For production, use secure methods to store
#     and access your credentials. For more information, see
#     https://docs.microsoft.com/en-us/azure/cognitive-services/cognitive-services-security?tabs=command-line%2Ccsharp#environment-variables-and-application-configuration
#     """
#     img_location = results['img_location']
#     results['applied_fuctions'].append('extract_from_doc')

#     # Call Azure AI Document Intelligence API --------------------------
#     client = DocumentAnalysisClient(
#         endpoint=ENDPOINT_DOCUMENT_INTELLIGENCE, 
#         credential=AzureKeyCredential(KEY_DOCUMENT_INTELLIGENCE)
#     )
#     if img_is_url:
#         response = client.begin_analyze_document_from_url(
#             model_id=MODEL_ID, 
#             document_url=img_location
#         )
    
#     else:
#         with open(img_location, 'rb') as f:
#             image_data = f.read()
            
#             response = client.begin_analyze_document(
#                 model_id=MODEL_ID, 
#                 document=image_data
#             )
#     # ------------------------------------------------------------------
    
#     fields = response.result().documents[0].fields
#     plate_confidence = fields['Placa'].confidence
#     vin_confidence   = fields['VIN'].confidence
#     # error_message = 'ERROR WITH'

#     # if plate_confidence >= THRESHOLD_DOC_EXTRACTION:
#     results['plate']['text'] = fields['Placa'].value
#     results['plate']['probability'] = plate_confidence
#     # else:
#         # error_message += ' PLATE'

#     # if vin_confidence >= THRESHOLD_DOC_EXTRACTION:
#     results['vin']['text'] = fields['VIN'].value
#     results['vin']['probability'] = vin_confidence
#     # else:
#         # error_message += ' VIN'

#     # if not error_message:
#     results['status'] = 'TEST'
#     # else:
#     #     results['status'] = 'REPEAT'
#     #     results['message'] = error_message

#     return results


# # extract_from_doc(IMG_PATH, img_is_url=False)

# Logic

In [ ]:
# import base64
# from PIL import Image
# from io import BytesIO

# def base64_to_image(base64_string):
#     # Decode the base64 string to bytes
#     img_data = base64.b64decode(base64_string)
#     # Create a BytesIO object from the bytes
#     img_buffer = BytesIO(img_data)
#     # Open the image using Pillow
#     image = Image.open(img_buffer)
#     return image

# def display_image(image):
#     image.show()

# # Example base64 string (replace with your actual base64 string)
# # base64_string = 'iVBORw0KGgoAAAANSUhEUgAAAAUA...'

# # Convert base64 string to image
# image = base64_to_image(base64_string)

# # Display the image
# display_image(image)

In [ ]:
import io
with open(IMG_PATH, 'rb') as img_binary:
    input_request = {
        'files': [img_binary]
    }
    print(io.FileIO(img_binary.fileno()))

In [ ]:
HEADERS_CUSTOM_VISION = {
    'Content-Type': 'application/octet-stream',
    'Prediction-Key': '5ceef9f635804c05ae36fdca6dd99eaf'
}

PROJECT_NAME = 'Iteration3%20%7C%20F2M2'
PROJECT_ID = '077a7a83-5a6f-4a47-a946-374814e786f4'
ENDPOINT_CLASSIFICATION = (
    f'https://cstvtclasificadorimagentesdeveastus2-prediction.'
    f'cognitiveservices.azure.com/customvision/v3.0/Prediction/'
    f'{PROJECT_ID}/classify/iterations/{PROJECT_NAME}/image'
)

import requests
IMG_PATH = 'work-zone/Fase2/Automóvil/FRONTAL/1BLSR5_RFFRE.jpg'

class Example:
    def __init__(self, img_binary):
        self.img_binary = img_binary
        self.content = {'applied_fuctions': []}
    
    def foo(self):
        try:
            response = requests.request( 
                method='POST', 
                url=ENDPOINT_CLASSIFICATION,
                headers=HEADERS_CUSTOM_VISION, 
                data=self.img_binary
            )
            self.content['applied_fuctions'].append('classify_vehicle_photo')
            response.raise_for_status()

            return response
        
        except Exception as x:
            raise x
    
    def classify_vehicle_photo(self):
        """
        Classifies an image using the Azure AI Custom Vision API and updates 
        the self.content dictionary with the most probable predictions.
        """
        self.content['applied_fuctions'].append('classify_vehicle_photo')

        # Call Azure AI Custom vision API (multi-label) ----------------
        response = requests.request( 
            method='POST', 
            url=ENDPOINT_CLASSIFICATION,
            headers=HEADERS_CUSTOM_VISION, 
            data=self.img_binary#img_binary
        )

        print(response.json())

# pprint(response.json())
with open(IMG_PATH, 'rb') as img_binary:
    input_request = {
        'files': [img_binary]
    }
    # ex = Example(input_request['files'][0])
    img = Example(input_request['files'][0])
    img.classify_vehicle_photo()
    # response = ex.foo()
img.content

In [ ]:
class VehicleImage:

    def __init__(self, img_binary):
        self.img_binary = img_binary
        self.content = {
            'status'          : None,
            'img_location'    : img_binary,
            'applied_fuctions': [],
            'n_info'          : 0,
            'vehicle_type'    : {'label': None, 'probability': None},
            'photo_type'      : {'label': None, 'probability': None},
            'vin'             : {'text' : None, 'probability': None},
            'plate'           : {'text' : None, 'probability': None}
        }


    def classify_vehicle_photo(self):
        """
        Classifies an image using the Azure AI Custom Vision API and updates 
        the self.content dictionary with the most probable predictions.
        """
        self.content['applied_fuctions'].append('classify_vehicle_photo')

        # Call Azure AI Custom vision API (multi-label) ----------------
        response = requests.request( 
            method='POST', 
            url=ENDPOINT_CLASSIFICATION,
            headers=HEADERS_CUSTOM_VISION, 
            data=self.img_binary#img_binary
        )

        # The predictions are ordered from lowest to highest probability 
        # to guarantee that the most probable values will be present in 
        # 'self.content', regardless of the number of labels predicted. 
        predictions = response.json()['predictions'][::-1]

        for prediction in predictions:  
                
            if prediction['probability'] >= THRESHOLD_CLASSIFICATION:

                if prediction['tagName'] in VEHICLE_TYPES:
                    self.content['vehicle_type']['label'] = \
                        prediction['tagName']
                    self.content['vehicle_type']['probability'] = \
                        prediction['probability']

                else: # PHOTO_TYPES
                    self.content['photo_type']['label'] = \
                        prediction['tagName'].split()[0]                # i.e. 'FRONTAL R4' -> 'FRONTAL'
                    self.content['photo_type']['probability'] = \
                        prediction['probability']

                self.content['n_info'] += 1
    
with open(IMG_PATH, 'rb') as img_binary:
    input_request = {
        'files': [img_binary]
    }

    img_example = VehicleImage(input_request['files'][0])
    img_example.classify_vehicle_photo()

pprint(img_example.content, width=200, sort_dicts=False)

In [ ]:
img_example.content

In [ ]:
import os

import requests

# Azure Services
from azure.ai.vision.imageanalysis import ImageAnalysisClient
from azure.ai.vision.imageanalysis.models import VisualFeatures
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.core.credentials import AzureKeyCredential

# Visualization
from PIL import Image
import matplotlib.pyplot as plt

#%% PARAMS
VEHICLE_TYPES = ['Automóvil', 'Camión', 'Motocicleta']
PHOTO_TYPES = [
    'FRONTAL R2'  , 'FRONTAL R4'  ,
    'TRASERA R2'  , 'TRASERA R4'  ,
    'DERECHA R2'  , 'DERECHA R4'  ,
    'IZQUIERDA R2', 'IZQUIERDA R4',
    'INTERIOR R2' , 'INTERIOR R4' ,
    'VIN',
    'DOCUMENTO'
]


#%% Azure APIs
# Custom Vision --------------------------------------------------------
HEADERS_CUSTOM_VISION = {
    'Content-Type': 'application/octet-stream',
    'Prediction-Key': '5ceef9f635804c05ae36fdca6dd99eaf'
}
# Classification
PROJECT_NAME = 'Iteration3%20%7C%20F2M2'
PROJECT_ID = '077a7a83-5a6f-4a47-a946-374814e786f4'
ENDPOINT_CLASSIFICATION = (
    f'https://cstvtclasificadorimagentesdeveastus2-prediction.'
    f'cognitiveservices.azure.com/customvision/v3.0/Prediction/'
    f'{PROJECT_ID}/classify/iterations/{PROJECT_NAME}/image'
)
THRESHOLD_CLASSIFICATION = 0.7
# Object Detection
PROJECT_NAME = 'Iteration2%20%7C%20F2M1'
PROJECT_ID = 'ca4fe2cf-23ff-44ab-8604-9a3b092caecf'
ENDPOINT_PLATE_DETECTION = (
    f'https://cstvtclasificadorimagentesdeveastus2-prediction.'
    f'cognitiveservices.azure.com/customvision/v3.0/Prediction/'
    f'{PROJECT_ID}/detect/iterations/{PROJECT_NAME}/image'
)
THRESHOLD_PLATE_DETECTION = 0.94
# ----------------------------------------------------------------------


# Computer Vision ------------------------------------------------------
ENDPOINT_PLATE_EXTRACTION = (
    'https://cv-clasificadorimagenes-dev-eastus2-01.'
    'cognitiveservices.azure.com/'
)
KEY_PLATE_EXTRACTION = 'a25a975c79e1418cbde0a23b4021b74e'
# ----------------------------------------------------------------------


# Document Intelligence ------------------------------------------------
ENDPOINT_DOCUMENT_INTELLIGENCE = 'https://eastus2.api.cognitive.microsoft.com/'
KEY_DOCUMENT_INTELLIGENCE = '1bcc168accb740f09a59b7313db475a4'
MODEL_ID = 'Iteracion2-neural'

THRESHOLD_DOC_EXTRACTION = 0.5
# ----------------------------------------------------------------------


#%% Vision Toolkit - Vehicle Image Class
class VehicleImage:

    def __init__(self, img_binary):
        self.img_binary = img_binary
        self.content = {
            'status'          : None,
            'img_location'    : img_binary,
            'applied_fuctions': [],
            'n_info'          : 0,
            'vehicle_type'    : {'label': None, 'probability': None},
            'photo_type'      : {'label': None, 'probability': None},
            'vin'             : {'text' : None, 'probability': None},
            'plate'           : {'text' : None, 'probability': None}
        }


    def classify_vehicle_photo(self):
        """
        Classifies an image using the Azure AI Custom Vision API and updates 
        the self.content dictionary with the most probable predictions.
        """
        self.content['applied_fuctions'].append('classify_vehicle_photo')

        # Call Azure AI Custom vision API (multi-label) ----------------
        response = requests.request( 
            method='POST', 
            url=ENDPOINT_CLASSIFICATION,
            headers=HEADERS_CUSTOM_VISION, 
            data=self.img_binary#img_binary
        )
        print(response.json())
        # --------------------------------------------------------------

        # The predictions are ordered from lowest to highest probability 
        # to guarantee that the most probable values will be present in 
        # 'self.content', regardless of the number of labels predicted. 
        # predictions = response.json()['predictions'][::-1]

        # for prediction in predictions:  
                
        #     if prediction['probability'] >= THRESHOLD_CLASSIFICATION:

        #         if prediction['tagName'] in VEHICLE_TYPES:
        #             self.content['vehicle_type']['label'] = \
        #                 prediction['tagName']
        #             self.content['vehicle_type']['probability'] = \
        #                 prediction['probability']

        #         else: # PHOTO_TYPES
        #             self.content['photo_type']['label'] = \
        #                 prediction['tagName'].split()[0]                # i.e. 'FRONTAL R4' -> 'FRONTAL'
        #             self.content['photo_type']['probability'] = \
        #                 prediction['probability']

        #         self.content['n_info'] += 1



# from vision_toolkit import VehicleImage
# from pprint import pprint
# IMG_PATH = 'work-zone/Fase2/NEGATIVO/Tests/TEST FIAT - FRONTAL CALIDAD MEDIA-ALTA.jpeg'
# with open(IMG_PATH, 'rb') as img_binary:
#     input_request = {
#         'files': [img_binary]
#     }


#     # Load image
#     print(input_request['files'][0])
#     img = VehicleImage(img_binary)
#     img.show()

#     img.classify_vehicle_photo()

#     pprint(img.content, width=200, sort_dicts=False)

In [ ]:
# IMG_PATH = 'work-zone/Fase2/Automóvil/FRONTAL/1BLSR5_RFFRE.jpg'
# IMG_PATH = 'work-zone/Fase2/NEGATIVO/TEST FIAT - FRONTAL CALIDAD MEDIA-ALTA.jpeg'
# IMG_PATH = 'work-zone/Fase2/NEGATIVO/TEST FIAT - VIN CALIDAD MEDIA-ALTA.jpeg' # LIM
# IMG_PATH = 'work-zone/Fase2/Automóvil/VIN/YTRMMD_RFVIN.jpg'
# IMG_PATH = 'work-zone/Fase2/Camión/TRASERA/WOS755 - TRASERA.png'
# IMG_PATH = 'work-zone/Fase2/Camión/INTERIOR/WZD418 - INTERIOR.jpeg'

# VIN
# IMG_PATH = 'work-zone/Fase2/NEGATIVO/Tests/TEST FIAT - VIN CALIDAD ALTA.jpeg'

#DOCUMENT
# IMG_PATH = 'work-zone/Fase2/DOCUMENTO/VXQ671_RFRTV.jpg'                 # Plate/VIN
# IMG_PATH = 'work-zone/Fase2/DOCUMENTO/xx2 - DOCUMENTO DEKRA.png'        # Only VIN
# IMG_IS_URL = False

In [ ]:
import base64
from PIL import Image
from io import BytesIO

def image_to_base64(image_path):
    # Open the image file
    with Image.open(image_path) as image:
        # Create a BytesIO object
        buffered = BytesIO()
        # Save the image to the BytesIO object in PNG format
        image.save(buffered, format='PNG')
        # Get the byte data from the BytesIO object
        img_byte = buffered.getvalue()
        # Encode the byte data to base64
        img_base64 = base64.b64encode(img_byte)
        # Convert base64 bytes to string
        img_base64_string = img_base64.decode('utf-8')
        return img_base64_string

# Example usage
image_path = IMG_PATH
base64_string = image_to_base64(image_path)
# print(base64_string)
print(len(base64_string))

In [ ]:
del VehicleImage

In [1]:
from vision_toolkit import VehicleImage
from pprint import pprint

IMG_PATH = 'work-zone/Fase2/NEGATIVO/Tests/TEST FIAT - VIN CALIDAD ALTA.jpeg' # VIN
# IMG_PATH = 'work-zone/Fase2/DOCUMENTO/VXQ671_RFRTV.jpg'                 # DOCUMENTO: Plate/VIN
# IMG_PATH = 'work-zone/Fase2/Automóvil/FRONTAL/1BLSR5_RFFRE.jpg'         # PLATE
# IMG_PATH = 'work-zone/Fase2/Camión/INTERIOR/WZD418 - INTERIOR.jpeg'     # INTERIOR Camión
# IMG_PATH = 'work-zone/Fase2/Motocicleta/FRONTAL/WLF34E - FRONTAL.jpeg'  # FRONTAL Motocicleta

with open(IMG_PATH, 'rb') as img_binary:
    input_request = {
        'files': [img_binary]
    }

    # Load image
    img = VehicleImage(img_binary)
    img.show()

    # Logic
    img.classify_vehicle_photo()                                        # [NEXT] extract_text, option cropped

    # No classification (likely due to image quality) -> REPEAT
    if (img.content['n_info'] == 0):
        img.content['status'] = 'REPEAT'

    # Classification 1 (VIN, DOCUMENTO): Get VIN or Plate -> ???
    if img.content['n_info'] == 1:
        if img.content['photo_type']['label'] == 'VIN':
            img.extract_text(content_type='vin')                            # [NEXT] clean text / URL
        # if img.content['photo_type']['label'] == 'DOCUMENTO':
        #     img.extract_entities_from_doc()                                 # [NEXT] use confidence

    # # Classification 2 (FRONTAL R2, TRASERA R2): Get plata -> ???
    # if img.content['n_info'] == 2:
    #     if (img.content['photo_type']['label'] in ['FRONTAL', 'TRASERA']):  # [NEXT] MOTO
    #         cropped_image = img.detect_plate()                              # [NEXT] don't save cropped img / URL
    #     else:
    #         img.content['status'] = 'SUCCESS'
    # # Classification 3+: Model error -> REVIEW
    # if (img.content['n_info'] >= 3):
    #     img.content['status'] = 'REVIEW'

pprint(img.content, width=200, sort_dicts=False)

<_io.BufferedReader name='work-zone/Fase2/NEGATIVO/Tests/TEST FIAT - VIN CALIDAD ALTA.jpeg'> <class '_io.BufferedReader'>


HttpResponseError: (InvalidRequest) The image size is not allowed to be zero or larger than 20971520 bytes.
Code: InvalidRequest
Message: The image size is not allowed to be zero or larger than 20971520 bytes.

In [ ]:
import matplotlib.pyplot as plt

def display_image(img_binary):
    img = plt.imread(img_binary)
    plt.imshow(img)
    plt.axis('off')
    plt.show()

with open(IMG_PATH, 'rb') as f:
    display_image(f)

In [ ]:
test_img = open(IMG_PATH, 'rb')
test_img

In [ ]:
# # Setup input
# IMG_IS_URL = False

# results = deepcopy(TEMPLATE)
# results['img_location'] = IMG_PATH

# # Classification 
# results = classify_image(                                               # DONE 
#     results=results,
#     img_is_url=IMG_IS_URL
# )

# if (results['n_info'] == 0):                                            # DONE
#     results['status'] = 'REPEAT'
#     print('REPEAT')

# if results['n_info'] == 1:
#     if results['photo_type']['label'] == 'VIN':
#         results = extract_text(
#             content_type='vin',
#             results=results, 
#             img_is_url=IMG_IS_URL
#         )
#         print('VIN')

#     if results['photo_type']['label'] == 'DOCUMENTO':                   # TEST
#         results = extract_from_doc(
#             results=results,
#             img_is_url=IMG_IS_URL
#         )
#         print('DOCUMENTO')

# if results['n_info'] == 2:
#     results, cropped_image = detect_plate(                              # NONE
#         img_location=IMG_PATH, 
#         results=results,
#         img_is_url=IMG_IS_URL
#     )
#     # Extract plate                                                     # NEXT
#     print('OTHER')

# if (results['n_info'] >= 3):
#     results['status'] = 'REVIEW'
#     print('REVIEW')

# pprint(results, width=10, sort_dicts=False)

In [ ]:
import unicodedata
import re

text = """
ÁÀÁÂB'BBBB/BBBBBB3B
"""
# Match:
# 9BD363B32PYZ78594
# 9BD3'63B32P'YZ78594
# 9B333676738748898
# 9B33367673/874/8898
# ÁÀÁÂBBBBBBBBBBB3B

# Don't match:
# 9BD363B32PYZ7859
# LONGWORDDDDDDDDD

In [1]:
from unidecode import unidecode

ModuleNotFoundError: No module named 'unidecode'

In [15]:
xx = 'hola      andres'
xx.strip()

'hola      andres'

In [16]:
import unicodedata
text = """
LCOCEADB220000006/LCOCEAD0220000096
¬|°*+~¨`"ÁÀÁÂB'BBBB/BBB\\BBB3B
"""
text_without_accents = [
    # Po (Other_Punctuation): "'/*
    # Sk ()
    {unicodedata.category(char): char} for char in text 
    if unicodedata.category(char) not in ['Po']
]
# set([list(xx.values())[0] for xx in text_without_accents])
text_without_accents

[{'Cc': '\n'},
 {'Lu': 'L'},
 {'Lu': 'C'},
 {'Lu': 'O'},
 {'Lu': 'C'},
 {'Lu': 'E'},
 {'Lu': 'A'},
 {'Lu': 'D'},
 {'Lu': 'B'},
 {'Nd': '2'},
 {'Nd': '2'},
 {'Nd': '0'},
 {'Nd': '0'},
 {'Nd': '0'},
 {'Nd': '0'},
 {'Nd': '0'},
 {'Nd': '0'},
 {'Nd': '6'},
 {'Lu': 'L'},
 {'Lu': 'C'},
 {'Lu': 'O'},
 {'Lu': 'C'},
 {'Lu': 'E'},
 {'Lu': 'A'},
 {'Lu': 'D'},
 {'Nd': '0'},
 {'Nd': '2'},
 {'Nd': '2'},
 {'Nd': '0'},
 {'Nd': '0'},
 {'Nd': '0'},
 {'Nd': '0'},
 {'Nd': '0'},
 {'Nd': '9'},
 {'Nd': '6'},
 {'Cc': '\n'},
 {'Sm': '¬'},
 {'Sm': '|'},
 {'So': '°'},
 {'Sm': '+'},
 {'Sm': '~'},
 {'Sk': '¨'},
 {'Sk': '`'},
 {'Lu': 'Á'},
 {'Lu': 'À'},
 {'Lu': 'Á'},
 {'Lu': 'Â'},
 {'Lu': 'B'},
 {'Lu': 'B'},
 {'Lu': 'B'},
 {'Lu': 'B'},
 {'Lu': 'B'},
 {'Lu': 'B'},
 {'Lu': 'B'},
 {'Lu': 'B'},
 {'Lu': 'B'},
 {'Lu': 'B'},
 {'Lu': 'B'},
 {'Nd': '3'},
 {'Lu': 'B'},
 {'Cc': '\n'}]

In [ ]:
text = "ÅngsÁÀtröm123 hola\n"
text_without_accents = ''.join(
    char for char in normalized_text 
    if unicodedata.category(char) != 'Mn'
)
text_without_accents

In [ ]:
img.info

In [17]:
import re



'\nLCOCEADB220000006/LCOCEAD0220000096\n¬|°*+~¨`"ÁÀÁÂB\'BBBB/BBB\\BBB3B\n'
'LCOCEADB220000006LCOCEAD0220000096AAAABBBBBBBBBBB3B'


In [ ]:
normalized_text = unicodedata.normalize('NFD', text)
    
# Remove accents by filtering out combining diacritical marks
text_without_accents = ''.join(
    char for char in normalized_text 
    if unicodedata.category(char) != 'Mn'
)

# Remove special characters by keeping only alphanumeric characters
cleaned_text = re.sub(r'[^A-Za-z0-9]', '', text_without_accents)

In [ ]:
# xx = eval(results['vin']['text'])
for x in xx:
    print()
    print(x['text'])
    print('\t', [word['text'] for word in x['words']])
    print('\t', [word['confidence'] for word in x['words']])

# Save Results

In [ ]:
import os
import pandas as pd

root_dir = 'work-zone\\Fase2'
data = {'vehicle_type': [], 'photo_type': [], 'img_path': []}


for root, dirs, imgs in os.walk(root_dir):

    vehicle_type = os.path.basename(os.path.dirname(root))
    photo_type = os.path.basename(root)
    
    if vehicle_type.istitle():
        for img in imgs:
            img_path = os.path.join(root, img)
            
            data['vehicle_type'].append(vehicle_type)
            data['photo_type'].append(photo_type)
            data['img_path'].append(img_path)
    else:
        print(vehicle_type)
        

# Create a dataframe from the dictionary
df = pd.DataFrame(data)

# Display the dataframe
df.groupby(['vehicle_type', 'photo_type']).count()

In [ ]:
img_info = df['img_path'].str.extract(r'\\(?P<id>[A-Z0-9]+)(?P<id_type>\s-\s|_)')
img_info['id_type'] = img_info['id_type'].replace({'_': 'FASE1', ' - ': 'FASE2'})
# img_info#.groupby('id').count()

In [ ]:
cond_document = (df['photo_type'] == 'DOCUMENTO')
df[cond_document]

In [ ]:
cond_plate_r2 = (df['vehicle_type'] == 'Motocicleta') &\
                (df['photo_type'] == 'TRASERA') 
cond_plate_r4 = df['vehicle_type'].isin(['Automóvil', 'Camión']) &\
                df['photo_type'  ].isin(['FRONTAL', 'TRASERA'])
cond_plate    = cond_plate_r2 | cond_plate_r4

df[cond_plate].groupby(['vehicle_type', 'photo_type']).count()